In [ ]:
import numpy as np
import torch
import torch.nn as nn

from nets import GraphNN_KNN_v1, EdgeClassifier_v1
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, accuracy_score, average_precision_score
from torch_geometric.data import DataLoader
from preprocessing import preprocess_dataset
from clustering_metrics import class_disbalance_graphx, class_disbalance_graphx__
from clustering_metrics import estimate_e, estimate_start_xyz, estimate_txty
from tqdm import tqdm


def predict_one_shower(shower, graph_embedder, edge_classifier):
    embeddings = graph_embedder(shower)
    edge_labels_true = (shower.y[shower.edge_index[0]] == shower.y[shower.edge_index[1]]).view(-1)
    edge_data = torch.cat([
        embeddings[shower.edge_index[0]],
        embeddings[shower.edge_index[1]]
    ], dim=1)
    edge_labels_predicted = edge_classifier(edge_data).view(-1)

    return edge_labels_true, edge_labels_predicted

In [ ]:
#!pip uninstall torch -y

In [ ]:
#!conda install pytorch==1.4.0 torchvision==0.5.0 cudatoolkit=10.1 -c pytorch -y

In [ ]:
#!pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.4.0.html

In [ ]:
#!pip install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.4.0.html

In [ ]:
#!pip install torch-cluster==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.4.0.html

In [ ]:
#!pip install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.4.0.html

In [ ]:
#!pip uninstall torch-geometric -y


In [ ]:
#!pip install torch-geometric

In [ ]:
#!pip install hdbscan

In [2]:
import hdbscan
from hdbscan import plots as hdbscan_plot
import networkx as nx
from clustering import preprocess_torch_shower_to_nx
import seaborn as sns
import matplotlib.pyplot as plt

min_cl = 40
cl_size = min_cl

In [3]:
datafile='clusters_rand.pt'
device = torch.device("cpu")

In [211]:
clusterized_bricks = torch.load(datafile)

In [212]:
def class_disbalance_graphx(graphx):
    signal = []
    for _, node in graphx.nodes(data=True):
        signal.append(node['signal'])
    return list(zip(*np.unique(signal, return_counts=True)))

In [216]:
selected_tracks = 0
total_tracks = 0
E = []
E_true_all = []
n_showers = []

total_number_of_showers = 0
number_of_lost_showers = 0
second_to_first_ratios = 0
number_of_good_showers = 0
number_of_stucked_showers = 0
number_of_broken_showers = 0

second_to_first_ratios = []
x_raw = []
x_true = []

y_raw = []
y_true = []

z_raw = []
z_true = []

tx_raw = []
tx_true = []

ty_raw = []
ty_true = []
n_showers = []

e_stucked_10_list = []
e_stucked_30_list = []
e_stucked_50_list = []

e_broken_10_list = []
e_broken_30_list = []
e_broken_50_list = []

e_good_10_list = []
e_good_30_list = []
e_good_50_list = []

for clusterized_brick in clusterized_bricks:
    showers_data = clusterized_brick['graphx'].graph['showers_data']
    print(len(showers_data))

    clusters = clusterized_brick['clusters']
    raw_clusters = clusterized_brick['raw_clusters']
    print('N predicted showers per brick:', len(raw_clusters))

    for shower_data in showers_data:
        shower_data['clusters'] = []
        shower_data['raw_clusters'] = []
        
    for cluster, raw_cluster in zip(clusters, raw_clusters):
        selected_tracks += len(cluster)
        for label, label_count in class_disbalance_graphx(cluster):
            if label_count / showers_data[label]['numtracks'] >= 0.1:
                showers_data[label]['clusters'].append(cluster)
                showers_data[label]['raw_clusters'].append(raw_cluster)
            
    for shower_data in showers_data:
        total_tracks += shower_data['numtracks']
        E.append(shower_data['numtracks'])
        E_true_all.append(shower_data['ele_P'])
        n_showers.append(len(showers_data)) 
        
    for shower_data in showers_data:
            total_number_of_showers += 1

            signals_per_cluster = []
            signals_per_cluster_bad = []
            idx_cluster = []
            for i, cluster in enumerate(shower_data['clusters']):
                labels, counts = class_disbalance_graphx__(cluster)           
                signals_per_cluster.append(counts[labels == shower_data['signal']][0])
                idx_cluster.append(i)
            signals_per_cluster = np.array(signals_per_cluster)
            
            
            
            if len(signals_per_cluster) == 0:
                number_of_lost_showers += 1               
                continue
            if len(signals_per_cluster) == 1:
                second_to_first_ratio = 0.
                second_to_first_ratios.append(second_to_first_ratio)
            else:
                second_to_first_ratio = np.sort(signals_per_cluster)[-2] / signals_per_cluster.max()
                second_to_first_ratios.append(second_to_first_ratio)

          

            cluster = shower_data['clusters'][np.argmax(signals_per_cluster)]
            
            

            # not enough signal
            if (signals_per_cluster.max() / shower_data['numtracks']) <= 0.1:
                continue

            labels, counts = class_disbalance_graphx__(cluster)
            counts = counts / counts.sum()
            # high contamination
            if counts[labels == shower_data['signal']] < 0.9:
                number_of_stucked_showers += 1
                #cluster = shower_data['clusters'][0]
                e_stucked_10 = estimate_e(cluster, angle=0.1)
                e_stucked_10_list.append(e_stucked_10)
                
                e_stucked_30 = estimate_e(cluster, angle=0.3)
                e_stucked_30_list.append(e_stucked_30)
                
                e_stucked_50 = estimate_e(cluster, angle=0.5)
                e_stucked_50_list.append(e_stucked_50)
                
                #print('stuck', shower_data['raw_clusters'])             
                #stability_stucked.append(shower_data['raw_clusters'].stability)
                continue

            if second_to_first_ratio > 0.3:
                number_of_broken_showers += 1
                #cluster = shower_data['clusters'][0]
                
                e_broken_10 = estimate_e(cluster, angle=0.1)
                e_broken_10_list.append(e_broken_10)
                
                e_broken_30 = estimate_e(cluster, angle=0.3)
                e_broken_30_list.append(e_broken_30)
                
                e_broken_50 = estimate_e(cluster, angle=0.5)
                e_broken_50_list.append(e_broken_50)
                
                #print('broken', shower_data['raw_clusters'])
                #stability_broken.append(shower_data['raw_clusters'][0].stability)
                continue
                
            

            # for good showers
            number_of_good_showers += 1

            
            # x, y, z
            x, y, z = estimate_start_xyz(cluster)
            
            e_good_10 = estimate_e(cluster, angle=0.1)
            e_good_30 = estimate_e(cluster, angle=0.3)
            e_good_50 = estimate_e(cluster, angle=0.5)

            e_good_10_list.append(e_good_10)
            e_good_30_list.append(e_good_30)
            e_good_50_list.append(e_good_50)
            
            #print('good', shower_data['raw_clusters'])
            #stability_good.append(shower_data['raw_clusters'][0].stability)

            x_raw.append(x)
            x_true.append(shower_data['ele_SX'])

            y_raw.append(y)
            y_true.append(shower_data['ele_SY'])

            z_raw.append(z)
            z_true.append(shower_data['ele_SZ'])

            # tx, ty
            tx, ty = estimate_txty(cluster)

            tx_raw.append(tx)
            tx_true.append(shower_data['ele_TX'])

            ty_raw.append(ty)
            ty_true.append(shower_data['ele_TY'])

       

238
N predicted showers per brick: 181
109
N predicted showers per brick: 71
132
N predicted showers per brick: 95
196
N predicted showers per brick: 143
162
N predicted showers per brick: 123
279
N predicted showers per brick: 196
138
N predicted showers per brick: 93
153
N predicted showers per brick: 120
226
N predicted showers per brick: 155
230
N predicted showers per brick: 176
189
N predicted showers per brick: 141
223
N predicted showers per brick: 171
192
N predicted showers per brick: 137
223
N predicted showers per brick: 164
265
N predicted showers per brick: 195
74
N predicted showers per brick: 58
234
N predicted showers per brick: 182
155
N predicted showers per brick: 109
189
N predicted showers per brick: 133
156
N predicted showers per brick: 114
275
N predicted showers per brick: 180
187
N predicted showers per brick: 132
175
N predicted showers per brick: 116
224
N predicted showers per brick: 152
66
N predicted showers per brick: 57
158
N predicted showers per bric

In [218]:
e_stucked_10_list = np.array(e_stucked_10_list)
e_stucked_30_list = np.array(e_stucked_30_list)
e_stucked_50_list = np.array(e_stucked_50_list)

e_broken_10_list = np.array(e_broken_10_list)
e_broken_30_list = np.array(e_broken_30_list)
e_broken_50_list = np.array(e_broken_50_list)

e_good_10_list = np.array(e_good_10_list)
e_good_30_list = np.array(e_good_30_list)
e_good_50_list = np.array(e_good_50_list)


In [222]:
e_10 = np.hstack((e_stucked_10_list, e_broken_10_list, e_good_10_list))

In [242]:
labels = np.hstack((np.array([0]*len(e_stucked_10_list)), np.array([1]*len(e_broken_10_list)),
                   np.array([2]*len(e_good_10_list))))

In [228]:
len(labels)==len(e_10)

True

In [229]:
e_30 = np.hstack((e_stucked_30_list, e_broken_30_list, e_good_30_list))

In [230]:
e_50 = np.hstack((e_stucked_50_list, e_broken_50_list, e_good_50_list))

In [235]:
data = np.vstack((e_10,e_30,e_50)).T

In [236]:
data.shape

(12564, 3)

In [238]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))

In [243]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.33, random_state=42)

In [244]:
clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [253]:
from sklearn.metrics import f1_score
f1_score(y_test, clf.predict(X_test), average = 'micro')

0.9787798408488063

------------------

In [ ]:
input_dim = showers[3].x.shape[1]
edge_dim = showers[3].edge_features.shape[1]
hidden_dim = 32
output_dim = 32
num_layers_emulsion = 5
num_layers_edge_conv = 3
min_samples_core = 4

graph_embedder = GraphNN_KNN_v1(
    output_dim=output_dim,
    hidden_dim=hidden_dim,
    edge_dim=edge_dim,
    num_layers_emulsion=num_layers_emulsion,
    num_layers_edge_conv=num_layers_edge_conv,
    input_dim=input_dim,
).to(device)
edge_classifier = EdgeClassifier_v1(
    input_dim=2 * output_dim + edge_dim,
).to(device)

In [ ]:
shower = showers[3]

G = preprocess_torch_shower_to_nx(
    shower,
    graph_embedder=graph_embedder,
    edge_classifier=edge_classifier,
    threshold=300,
    add_noise=0.,
    baseline=True
)

In [ ]:
tx_raw

In [ ]:
ty_raw

In [ ]:
connected_components = []
for cnn in nx.connected_components(nx.Graph(G)):
    if len(cnn) > min_cl:
        print(len(cnn), end=", ")
        connected_components.append(nx.DiGraph(G.subgraph(cnn)))
connected_component = connected_components[4]

## Viz linkage trees

In [ ]:
distance_matrix = nx.adjacency_matrix(nx.Graph(connected_component)).toarray().astype(np.float64)
distance_matrix[distance_matrix == 0.] = np.inf
clusterer = hdbscan.HDBSCAN(min_cluster_size=cl_size,
                            min_samples=min_samples_core,
                            metric="precomputed",
                            core_dist_n_jobs=-1)  # precomputed
clusterer.fit(distance_matrix);

In [ ]:
from custom_hdbscan import run_hdbscan
ewm_clusters, ewm_hdbscan, ewm_linkage = run_hdbscan(connected_component, cl_size=40)
ewm_linkage = np.array(ewm_linkage)

In [ ]:
sns.set(context='paper', style="whitegrid", font_scale=2)
plt.figure(figsize=(12, 8), dpi=100)
clusterer.single_linkage_tree_.plot()
#plt.savefig("hdbscan_single_linkage.pdf", bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
hdbscan.plots.SingleLinkageTree(ewm_linkage).plot()
#plt.savefig("ewm_single_linkage.pdf", bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
clusterer.condensed_tree_.plot(select_clusters=True,
                               selection_palette=sns.color_palette('deep', 8))
#plt.savefig("hdbscan_condensed.pdf", bbox_inches='tight')
plt.show()

In [ ]:
from hdbscan._hdbscan_tree import condense_tree
plt.figure(figsize=(12, 8))
hdbscan.plots.CondensedTree(condense_tree(ewm_linkage, 40)).plot(
    select_clusters=True,
    selection_palette=sns.color_palette('deep', 8)
)
#plt.savefig("ewn_condensed.pdf", bbox_inches='tight')
plt.show()

In [ ]:
#Here the parent denotes the id of the parent cluster, 
#the child the id of the child cluster (or, if the child is a single data point rather than a cluster, 
#the index in the dataset of that point), the lambda_val provides the lambda value at which the edge forms, 
#and the child_size provides the number of points in the child cluster. 

#As you can see the start of the DataFrame has singleton points falling out of the root cluster,
#with each child_size equal to 1.

In [ ]:
hdbscan.plots.CondensedTree(condense_tree(ewm_linkage, 40)).to_pandas()

In [ ]:
ewm_clusters[0]

## Viz graph itself

In [ ]:
AVAILABLE_COLORS = [
    '#e6194B', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#42d4f4', 
    'grey', '#469990', 'black', '#9A6324', '#fffac8', '#800000', 
    '#aaffc3', '#000075', '#a9a9a9', '#000000'
]
import copy
from celluloid import Camera
def plot_graphx(nodes: list, nodes_order: list, azim=-84, elev=10):
    """
    Function for plotting shower
    """
    from mpl_toolkits.mplot3d import Axes3D
    from mpl_toolkits.mplot3d.art3d import Line3DCollection
    import matplotlib.pyplot as plt
    dZ = 205. / 10000.
    fig = plt.figure(figsize=(12, 12))
    camera = Camera(fig)
    C = plt.cm.Blues(0.9)
    ax = fig.gca(projection='3d')
    # ax.set_xlabel("z")
    # ax.set_ylabel("y")
    # ax.set_zlabel("x") 
    # ax.set_xlim(z0.min(), z1.max())
    # ax.set_ylim(y0.min(), y1.max())
    #  ax.set_zlim(x0.min(), x1.max())

    x0, y0, z0 = [], [], []
    sx, sy = [], []
    colors = []
    AVAILABLE_COLORS
    signal = np.unique([n[1]["signal"] for n in nodes], return_counts=False)
    print(signal)
    map_signal_color = {}
    for i, s in enumerate(signal):
        map_signal_color[s] = AVAILABLE_COLORS[i]
    for i, node_id in enumerate(nodes_order):
        node = nodes[int(node_id)]
        node = node[1]
        if node["signal"] == -1:
            continue # skip noise nodes
        x0.append(node['features']['SX'])
        y0.append(node['features']['SY'])
        z0.append(node['features']['SZ'])
        sx.append(node['features']['TX'])
        sy.append(node['features']['TY'])
        colors.append(map_signal_color[node["signal"]])
        np.unique([n[1]["signal"] for n in nodes_new], return_counts=True)
        if i % 10 == 0:
            print(node_id)
            x0_np, y0_np, z0_np = np.array(x0), np.array(y0), np.array(z0)
            sx_np, sy_np = np.array(sx), np.array(sy)

            x1_np = x0_np + dZ * sx_np
            y1_np = y0_np + dZ * sy_np
            z1_np = z0_np + dZ

            start_points = np.array([z0_np, y0_np, x0_np]).T.reshape(-1, 3)
            end_points = np.array([z1_np, y1_np, x1_np]).T.reshape(-1, 3)

            lc = Line3DCollection(list(zip(start_points, end_points)), colors=colors, alpha=0.9, lw=2)
            ax.set_xlim(z0_np.min(), z1_np.max())
            ax.set_ylim(y0_np.min(), y1_np.max())
            ax.set_zlim(x0_np.min(), x1_np.max())
            ax.view_init(azim=azim, elev=elev)
            ax.add_collection3d(lc)
            camera.snap()
    animation = camera.animate()
    return animation

In [ ]:
connected_component_viz = copy.deepcopy(connected_component)
nodes = list(connected_component_viz.nodes(data=True))
edges = list(connected_component_viz.edges(data=True))

nodes_added = []
for q in ewm_linkage:
    if (not q[0] in nodes_added) and (q[0] < len(connected_component)):
        nodes_added.append(q[0])
    if (not q[1] in nodes_added) and (q[1] < len(connected_component)):
        nodes_added.append(q[1])
        
nodes_mapping = {}
nodes_new = []
for idx, node in enumerate(nodes):
    nodes_mapping[node[0]] = idx
    nodes_new.append((idx, node[1]))
edges_new = []
for idx, edge in enumerate(edges):
    edges_new.append(
        (nodes_mapping[edge[0]], nodes_mapping[edge[1]], edge[2])
    )
    
animation = plot_graphx(nodes_new, nodes_added, azim=-30, elev=20)
animation.save('ewm_clusterer.mp4', dpi=100,
               savefig_kwargs={
                   'frameon': False,
                   'pad_inches': 'tight'
               })

In [ ]:
connected_component_viz = copy.deepcopy(connected_component)
nodes = list(connected_component_viz.nodes(data=True))
edges = list(connected_component_viz.edges(data=True))

nodes_added = []
for q in clusterer.single_linkage_tree_.to_numpy():
    if (not q[0] in nodes_added) and (q[0] < len(connected_component)):
        nodes_added.append(q[0])
    if (not q[1] in nodes_added) and (q[1] < len(connected_component)):
        nodes_added.append(q[1])
        
nodes_mapping = {}
nodes_new = []
for idx, node in enumerate(nodes):
    nodes_mapping[node[0]] = idx
    nodes_new.append((idx, node[1]))
edges_new = []
for idx, edge in enumerate(edges):
    edges_new.append(
        (nodes_mapping[edge[0]], nodes_mapping[edge[1]], edge[2])
    )

animation = plot_graphx(nodes_new, nodes_added, azim=-30, elev=20)
animation.save('vanilla_clusterer.mp4', dpi=100,
               savefig_kwargs={
                   'frameon': False,
                   'pad_inches': 'tight'
               })

In [ ]:
connected_component_viz = copy.deepcopy(connected_component)
nodes = list(connected_component_viz.nodes(data=True))
edges = list(connected_component_viz.edges(data=True))

nodes_added = []
for q in ewm_linkage:
    if (not q[0] in nodes_added) and (q[0] < len(connected_component)):
        nodes_added.append(q[0])
    if (not q[1] in nodes_added) and (q[1] < len(connected_component)):
        nodes_added.append(q[1])
        
ewm_labels = np.full_like(clusterer.labels_, -1)
for c, cluster in enumerate(ewm_clusters):
    for node in cluster.nodes:
        ewm_labels[nodes_mapping[node]] = c
        
nodes_mapping = {}
nodes_new = []
for idx, node in enumerate(nodes):
    nodes_mapping[node[0]] = idx
    nodes_new.append((idx, node[1]))
    
ewm_labels = np.full_like(clusterer.labels_, -1)
for c, cluster in enumerate(ewm_clusters):
    for node in cluster.nodes:
        ewm_labels[nodes_mapping[node]] = c
        
for idx, node in enumerate(nodes_new):
    node[1]["signal"] = ewm_labels[idx]
    
edges_new = []
for idx, edge in enumerate(edges):
    edges_new.append(
        (nodes_mapping[edge[0]], nodes_mapping[edge[1]], edge[2])
    )
    
    
animation = plot_graphx(nodes_new, nodes_added, azim=-30, elev=20)
animation.save('ewm_clusterer_with_labels.mp4')

In [ ]:
connected_component_viz = copy.deepcopy(connected_component)
nodes = list(connected_component_viz.nodes(data=True))
edges = list(connected_component_viz.edges(data=True))

nodes_added = []
for q in clusterer.single_linkage_tree_.to_numpy():
    if (not q[0] in nodes_added) and (q[0] < len(connected_component)):
        nodes_added.append(q[0])
    if (not q[1] in nodes_added) and (q[1] < len(connected_component)):
        nodes_added.append(q[1])
        
nodes_mapping = {}
nodes_new = []
for idx, node in enumerate(nodes):
    nodes_mapping[node[0]] = idx
    nodes_new.append((idx, node[1]))
    node[1]["signal"] = clusterer.labels_[idx]
    
edges_new = []
for idx, edge in enumerate(edges):
    edges_new.append(
        (nodes_mapping[edge[0]], nodes_mapping[edge[1]], edge[2])
    )

animation = plot_graphx(nodes_new, nodes_added, azim=-30, elev=20)
animation.save('vanilla_clusterer_with_labels.mp4')